In [2]:
%pylab inline
import torch
import torch.nn.functional as F
import numpy as np
dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

Populating the interactive namespace from numpy and matplotlib


In [63]:
not_bothering = """i'd
we'd
lmao
lang=
&raquo
6.
they've
😂
they'd
t.co
you've
9)|!(ie)]>
@realdonaldtrump
f.d.a.
&amp
4.
he'd
y'all
<html
we've
7.
you'd
careerarc
8.
i've
:"""
not_bothering = set(not_bothering.split("\n"))

vocab = {}
inverted_vocab = {}
vocab_file = open("../nlp/idf.txt")

print not_bothering
i = 0
for line in vocab_file:
    word, _ = line.split()
    if word in not_bothering:
        continue
    vocab[word] = i
    inverted_vocab[i] = word
    i += 1


set(["y'all", '8.', "you'd", '6.', 'lang=', "we'd", "he'd", "we've", '4.', 'careerarc', 't.co', ':', '9)|!(ie)]>', "i've", '&amp', '7.', '<html', '@realdonaldtrump', "they'd", "you've", '\xf0\x9f\x98\x82', '&raquo', 'lmao', "they've", "i'd", 'f.d.a.'])


In [54]:
glove_vecs = {}
vec_file = open('../nlp/glove.6B.300d.txt')
for i, line in enumerate(vec_file):
    tline = line.strip().split()
    word, glove_vec = tline[0], np.array(tline[1:], dtype=np.float64)
    glove_vecs[word] = glove_vec / np.linalg.norm(glove_vec)


In [64]:
len(vocab)

6503

In [73]:
model = torch.nn.Sequential(
    torch.nn.Linear(len(vocab),4000),
    torch.nn.ReLU(),
    torch.nn.Linear(4000,2000),
    torch.nn.ReLU(),
    torch.nn.Linear(2000,1000),
    torch.nn.ReLU(),
    torch.nn.Linear(1000,500),  
    torch.nn.ReLU(),
    torch.nn.Linear(500,300)
)


In [113]:
squash = 30
model = torch.nn.Sequential(
    torch.nn.Linear(len(vocab)/squash,500),
    torch.nn.Sigmoid(),
    torch.nn.Linear(500,300)
)

x = np.identity(len(vocab)/squash)
print len(x)
y = [glove_vecs[inverted_vocab[i]] for i in range(len(vocab)/squash)]

x = torch.tensor(x, device=device, dtype = dtype)
y = torch.tensor(y, device=device, dtype = dtype)
loss_fn = torch.nn.MSELoss(reduction='sum')

optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum = 0.9)
for t in range(5000):

    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t%1000 == 0:
        print t, loss.item()

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
print t, loss.item()

216
0 6819.10449219
1000 124.179328918
2000 124.179328918
3000 124.179328918
4000 124.179328918
4999 124.179328918


In [118]:
squash = 30
model = torch.nn.Sequential(
    torch.nn.Linear(len(vocab)/squash,300),
    torch.nn.Sigmoid(),
    torch.nn.Linear(300,300)
)

x = np.identity(len(vocab)/squash)
print len(x)
y = [glove_vecs[inverted_vocab[i]] for i in range(len(vocab)/squash)]

x = torch.tensor(x, device=device, dtype = dtype)
y = torch.tensor(y, device=device, dtype = dtype)
loss_fn = torch.nn.MSELoss(reduction='sum')

optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum = 0.9)
for t in range(100000):

    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t%1000 == 0:
        print t, loss.item()

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
print t, loss.item()

216
0 5088.71386719
1000 99.673576355
2000 59.4380607605
3000 31.8491287231
4000 17.3755187988
5000 10.0559482574
6000 6.23392820358
7000 4.11217927933
8000 2.85438919067
9000 2.0633430481
10000 1.54046070576
11000 1.18066477776
12000 0.924973011017
13000 0.738370239735
14000 0.599068880081
15000 0.492989480495
16000 0.410759568214
17000 0.345982164145
18000 0.294201552868
19000 0.252259045839
20000 0.217876330018
21000 0.189386829734
22000 0.165552213788
23000 0.145440205932
24000 0.128339156508
25000 0.113698877394
26000 0.101089164615
27000 0.0901693180203
28000 0.0806667208672
29000 0.0723614320159
30000 0.0650737509131
31000 0.0586562529206
32000 0.0529865175486
33000 0.0479624010623
34000 0.0434982664883
35000 0.0395213589072
36000 0.0359701290727
37000 0.0327917858958
38000 0.0299414843321
39000 0.0273799225688
40000 0.0250738337636
41000 0.0229937490076
42000 0.0211143288761
43000 0.0194133631885
44000 0.0178715121001
45000 0.0164716970176
46000 0.015198957175
47000 0.014040059

In [116]:
squash = 30
model = torch.nn.Sequential(
    torch.nn.Linear(len(vocab)/squash,1000),
    torch.nn.Sigmoid(),
    torch.nn.Linear(1000,300)
)

x = np.identity(len(vocab)/squash)
print len(x)
y = [glove_vecs[inverted_vocab[i]] for i in range(len(vocab)/squash)]

x = torch.tensor(x, device=device, dtype = dtype)
y = torch.tensor(y, device=device, dtype = dtype)
loss_fn = torch.nn.MSELoss(reduction='sum')

optimizer = torch.optim.SGD(model.parameters(), lr=1e-5, momentum = 0.9)
for t in range(50000):

    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t%10 == 0:
        print t, loss.item()

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
print t, loss.item()

216
0 5525.52392578
1000 120.697769165
2000 115.051040649
3000 109.465446472
4000 103.833938599
5000 98.0824432373
6000 92.174118042
7000 86.1146774292
8000 79.9541702271
9000 73.7805252075
10000 67.7029876709
11000 61.8299407959
12000 56.2494773865
13000 51.0198059082
14000 46.1702804565
15000 41.7084579468
16000 37.6278572083
17000 33.9136428833
18000 30.5461578369
19000 27.5029010773
20000 24.7598876953
21000 22.2926120758
22000 20.0768203735
23000 18.0891036987
24000 16.3073043823
25000 14.7107553482
26000 13.2804260254
27000 11.9989528656
28000 10.8506002426
29000 9.82117843628
30000 8.89795303345
31000 8.06952381134
32000 7.32569885254
33000 6.6573882103
34000 6.05648517609
35000 5.51577234268
36000 5.02881717682
37000 4.58989715576
38000 4.19391727448
39000 3.83634090424
40000 3.51313304901
41000 3.22069716454
42000 2.95583510399
43000 2.71569681168
44000 2.49774575233
45000 2.29971861839
46000 2.11960053444
47000 1.95559513569
48000 1.80609810352
49000 1.66967713833
49999 1.545

In [154]:
squash = 1
model = torch.nn.Sequential(
    torch.nn.Linear(len(vocab)/squash,300),
)

x = np.identity(len(vocab)/squash)
print len(x)
y = [glove_vecs[inverted_vocab[i]] for i in range(len(vocab)/squash)]

x = torch.tensor(x, device=device, dtype = dtype)
y = torch.tensor(y, device=device, dtype = dtype)
loss_fn = torch.nn.MSELoss(reduction='sum')

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
for t in range(50):

    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t%1 == 0:
        print t, loss.item()

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()
model.eval()   
y_pred = model(x)
loss = loss_fn(y_pred, y)
print "final loss", loss.item()

6503
0 6701.97509766
1 6489.63085938
2 6285.89697266
3 6090.41064453
4 5902.72851562
5 5722.35253906
6 5548.80908203
7 5381.66552734
8 5220.52490234
9 5065.01904297
10 4914.81103516
11 4769.59667969
12 4629.10009766
13 4493.07080078
14 4361.28466797
15 4233.54736328
16 4109.69335938
17 3989.5793457
18 3873.07714844
19 3760.0715332
20 3650.45898438
21 3544.14404297
22 3441.03564453
23 3341.04516602
24 3244.08544922
25 3150.07226562
26 3058.92236328
27 2970.55444336
28 2884.88647461
29 2801.83642578
30 2721.32226562
31 2643.26416016
32 2567.58447266
33 2494.20825195
34 2423.06494141
35 2354.08544922
36 2287.20410156
37 2222.35620117
38 2159.47924805
39 2098.51269531
40 2039.39697266
41 1982.07385254
42 1926.48571777
43 1872.57666016
44 1820.29272461
45 1769.58227539
46 1720.39489746
47 1672.68200684
48 1626.39685059
49 1581.49414062
final loss 1537.93041992


After running on a GPU

where I used:

```
In [1]: torch.save(model.state_dict(), "backward_model")
```

In [160]:
model = torch.nn.Sequential(
    torch.nn.Linear(len(vocab),300),
)
model.load_state_dict(torch.load("backward_model", map_location='cpu'))
model.eval();

In [161]:
y_pred = model(x)
loss = loss_fn(y_pred, y)
print "final loss", loss.item()

final loss 0.000905576744117


Finally, it's time to try and reverse this process

In [173]:
w = torch.zeros((len(vocab), 300))
b = torch.zeros(300)
for name, param in model.named_parameters():
    if "bias" in name:
        b = param
    else:
        w = param

In [191]:
sq_weight = w.mm(w.t()).sum()
print sq_weight
for i in range(30):
    guess = w.t().mm((y[i] - b).view(300,-1))/ sq_weight
    print np.argmax(guess.tolist()) #softmax would work well

tensor(5022.0073, grad_fn=<SumBackward0>)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
